# Demo KQL Extension

Written by Taiob  Ali

Lexington Data Technology Group  

SqlWorldWide.com

# Connecting to Azure Data Explorer Cluster

- Change Kernel to '**Kusto**' 
- Select Connetion '**help.kusto.windows.net**'
- Select Database '**Samples**'

## Query data

In [ ]:
StormEvents
| where isnotempty(EndLocation) 
| summarize event_count=count() by EndLocation
| order by event_count

In [ ]:
StormEvents
| summarize event_count=count(), mid = avg(BeginLat) by State
| sort by mid
| where event_count > 1800
| project State, event_count

# Connecting to Log Analytics Workspace

## Ref: [Query data in Azure Monitor using Azure Data Explorer (Preview)](https://docs.microsoft.com/en-us/azure/data-explorer/query-monitor-data)

## Query data

Connection String:  '<span style="color: rgb(206, 145, 120); font-family: Consolas, &quot;Courier New&quot;, monospace; white-space: pre;">https://ade.loganalytics.io' + Resource Id of your log analytics workspace</span>

In [ ]:
cluster('https://ade.loganalytics.io/subscriptions/6f8db000-8416-43f0-a2db-cbfb7c945982/resourceGroups/sqlldtgdemo/providers/Microsoft.OperationalInsights/workspaces/sqlldtgws').database('sqlldtgws').AzureDiagnostics | take 10

In [ ]:
cluster('https://ade.loganalytics.io/subscriptions/6f8db000-8416-43f0-a2db-cbfb7c945982/resourceGroups/sqlldtgdemo/providers/Microsoft.OperationalInsights/workspaces/sqlldtgws').database('sqlldtgws').AzureDiagnostics 
| where LogicalServerName_s == "sqlpassdemoservertaiob"
| where TimeGenerated >= ago(5d)
| summarize count() by Category

In [ ]:
cluster('https://ade.loganalytics.io/subscriptions/6f8db000-8416-43f0-a2db-cbfb7c945982/resourceGroups/sqlldtgdemo/providers/Microsoft.OperationalInsights/workspaces/sqlldtgws').database('sqlldtgws').AzureMetrics
| where ResourceProvider == "MICROSOFT.SQL"
| where TimeGenerated >=ago(5d)
| where MetricName in ('deadlock')
| parse _ResourceId with * "/microsoft.sql/servers/" Resource // subtract Resource name for _ResourceId
| summarize Deadlock_max_60Mins = max(Maximum) by Resource, MetricName